<h1><strong>Stacking Classifier</strong> </h1> 
    
<h3>Acá se va a estar probando el ensamblaje de Stacking con los distintos algoritmos implementados.</h3>

Primero se hacen los imports necesarios

In [9]:
import pandas as pd
import numpy as np
import nbimporter # pip install nbimporter

import sklearn

from sklearn import svm
from catboost import CatBoostClassifier # pip install catboost
from lightgbm import LGBMClassifier  #pip install lightgbm
from xgboost import XGBClassifier #pip install xgboost
from sklearn.naive_bayes  import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import StackingClassifier, BaggingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix


from feature_builder import process_dataset
from Exporter import export_model_csv
from hyperparameter_tunning import obtener_hiperparametros

hyper_params = obtener_hiperparametros()

In [10]:
train_dataset = pd.read_csv('train.csv')

In [11]:
test_dataset = pd.read_csv('test.csv')

In [12]:
y = train_dataset.loc[:,'target']

<h2><strong>Preparo los distintos sets con features diferentes.</h2>
Los distintos algoritmos necesitan diferentes sets, según lo investigado.

Primero los procesados completos con spacy

In [13]:
x_processed = process_dataset(train_dataset, use_spacy=True)

Percentage of words covered in the embeddings = 0.4937444933920705
Percentage of words covered in the embeddings = 0.5961027457927369


In [14]:
x_train_processed, x_test_processed, y_train_processed, y_test_processed = train_test_split(x_processed, y, test_size = .30, random_state = 17)

Ahora los que solo necesitan embeddings. 

In [9]:
x_embedd = process_dataset(train_dataset, use_spacy=True, use_manual_features=False)

Percentage of words covered in the embeddings = 0.4937444933920705
Percentage of words covered in the embeddings = 0.5961027457927369


In [10]:
x_train_embedd, x_test_embedd, y_train_embedd, y_test_embedd = train_test_split(x_embedd, y, test_size = .30, random_state = 17)

<h2><strong> Defino los algoritmos a usar
    

In [15]:
logisticRegr = LogisticRegression(solver='liblinear', penalty='l1', multi_class='auto', max_iter=1000, C=1)

In [16]:
SVC = svm.SVC(degree=10,coef0=10,C=5, probability=True)

In [17]:
catboost = CatBoostClassifier(verbose=False)

In [10]:
MultiNB = MultinomialNB()

In [11]:
xgbooster = XGBClassifier(max_depth=3, n_estimators=600, colsample_bytree=0.9,
                        subsample=0.9, nthread=4, learning_rate=0.05)
BC_XGB = BaggingClassifier(base_estimator= xgbooster, n_estimators=10, random_state=0)

In [12]:
gbm = LGBMClassifier()
BC_LGB = BaggingClassifier(base_estimator= gbm, n_estimators=10, random_state=0)

<h1><strong> Empiezo las pruebas

------------------------------------------------------------------------------------------------------
<h3>PODRIAMOS PROBAR UNO QUE HAGA TODAS LAS COMBINACIONES Y DEVUELVA EL MEJOR, QUIEN SE ANIMA</h3>
----------------------------------------------------------------------------------------------------------------

<h3> Logistic, SVC, CatBoost, BC_XGB, BC_LGB

In [18]:
SC_LSC = StackingClassifier(estimators=[('lr', logisticRegr), ('svc', SVC), ('catboost', catboost),('bc_xgb',BC_XGB),('bc_lgb',BC_LGB)])

In [ ]:
SC_LSC.fit(x_train_processed, y_train_processed)

In [ ]:
y_pred_processed_LSC = SC_LSC.predict(x_test_processed)

In [ ]:
f1_score(y_test_processed, y_pred_processed_LSC)

<h3> Logistic, SVC, CatBoost

In [19]:
SC_LSC = StackingClassifier(estimators=[('lr', logisticRegr), ('svc', SVC), ('catboost', catboost)])

Pruebo procesadas

In [20]:
SC_LSC.fit(x_train_processed, y_train_processed)

StackingClassifier(cv=None,
                   estimators=[('lr',
                                LogisticRegression(C=1, class_weight=None,
                                                   dual=False,
                                                   fit_intercept=True,
                                                   intercept_scaling=1,
                                                   l1_ratio=None, max_iter=1000,
                                                   multi_class='auto',
                                                   n_jobs=None, penalty='l1',
                                                   random_state=None,
                                                   solver='liblinear',
                                                   tol=0.0001, verbose=0,
                                                   warm_start=False)),
                               ('svc',
                                SVC(C=5, break_ties=False, cache_size=200,
                       

In [21]:
y_pred_processed_LSC = SC_LSC.predict(x_test_processed)

In [22]:
f1_score(y_test_processed, y_pred_processed_LSC)

0.7951425554382259

<h3> Logistic, CatBoost

In [56]:
SC_LC = StackingClassifier(estimators=[('lr', logisticRegr), ('catboost', catboost)])

Pruebo procesadas

In [57]:
SC_LC.fit(x_train_processed, y_train_processed)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l1',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7f3591d35850>)],
                 flatten_transform=True, n_jobs=None, voting='hard',
                 weights=None)

In [58]:
y_pred_processed_LC = SC_LC.predict(x_test_processed)

In [59]:
f1_score(y_test_processed, y_pred_processed_LC)

0.7770915216170691

<h3> LogisticRegr, SVM, Catboost

In [43]:
SC_LSC = StackingClassifier(estimators=[('lr', logisticRegr), ('svc', SVC), ('catboost', catboost)])

In [44]:
AC_LSC.fit(x_train_processed, y_train_processed)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l1',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('svc',
                              SVC(C=5, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=10,
                                  decision_function_shape='ovr', degree=10,
                                  gamm

In [45]:
y_pred_processed_LSC = AC_LSC.predict(x_test_processed)

In [46]:
f1_score(y_test_processed, y_pred_processed_LSC)

0.7830744509908945

<h3>LR y CB

<h2>Paso a exportar el mejor resultado

In [23]:
x_train_final = process_dataset(train_dataset, use_spacy=True)

Percentage of words covered in the embeddings = 0.4937444933920705
Percentage of words covered in the embeddings = 0.5961027457927369


In [24]:
x_test_final = process_dataset(test_dataset, use_spacy=True)

Percentage of words covered in the embeddings = 0.5707598689343111
Percentage of words covered in the embeddings = 0.665389037945573


In [25]:
export_model_csv(SC_LSC, x_train_final, x_test_final, test_dataset, y, 'SC-LSC-processed.csv')